## Building Multi-Agent Systems with the AutoGen AgentChat API 

AutoGen recently released a new version of the [framework (v0.4)](https://microsoft.github.io/autogen/0.2/blog/) with an asynchronous event driven api where agents primary act when they receive a message. It introduces the concept of runtimes (which may be single threaded or distributed) and agents that can be run on these runtimes.

It also includes a high level API - [AgentChat]() that focuses on providing presets for agent behaviours and how these agents can interact as part of a team or workflow.

This notebook provides an example of the AgentChat API.



###   Migrating from AutoGen v0.2 -> AutoGen AgentChat v0.4 

AgentChat is written to be at similar level of abstraction as AutoGen 0.2
- UserProxyAgent -> UserProxyAgent
- AssistantAgent -> AssistantAgent  
- GroupChat 
   - RoundRobinGroupChat -> GroupChat (speaker_selection='round_robin') 
   - SelectorGroupChat -> GroupChat (speaker_selection='llm')  


In [27]:
# install the agentchat package
# !pip install -U 'autogen-agentchat==0.4.0.dev6' 'autogen-ext==0.4.0.dev6'  openai

In [28]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.task import Console, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_ext.models import OpenAIChatCompletionClient

## A Single Agent 

Basic example of a single agent that can receive a message and respond using an LLM."

In [29]:
agent = AssistantAgent( name="single_agent", model_client=OpenAIChatCompletionClient( model="gpt-4o-mini")) 

result = await agent.run(task="What is the height of the eiffel tower?") 
print(result.messages[-1].content)

The height of the Eiffel Tower is approximately 1,083 feet (330 meters) including its antennas. The tower itself, without antennas, is about 1,063 feet (324 meters) tall. TERMINATE


In [30]:
agent = AssistantAgent( name="single_agent", model_client=OpenAIChatCompletionClient( model="gpt-4o-mini")) 

result = await agent.run(task="What is the Weather in San Francisco?") 
print(result.messages[-1].content)

I can't provide real-time weather updates. I recommend checking a reliable weather website or app for the latest information on the weather in San Francisco. TERMINATE


In [31]:
def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

agent = AssistantAgent(
    name="basic_agent", model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"), 
    tools=[get_weather])
result = await agent.run(task="What is the Weather in San Francisco?") 
print(result.messages[-1].content)

The weather in San Francisco is 73 degrees and sunny.


## Defining an Agent Team 

AgentChat offers several team presents. In the following section, we will review a RoundRobinGroupChat team where each agent in the team takes turns to respond to a message in a round-robin fashion.

In [32]:
agent = AssistantAgent(
    name="basic_agent", model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"))

team = RoundRobinGroupChat(
    participants=[agent], termination_condition=TextMentionTermination("TERMINATE"))

team_result = await team.run(task="What is the weather in San Francisco?")


In [33]:
for message in team_result.messages:
    print(message.content)

What is the weather in San Francisco?
I cannot provide real-time data like current weather conditions. You can check a reliable weather website or a weather app for the latest updates on San Francisco's weather. TERMINATE


## Giving Agents Access to Tools 

In [34]:
def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

agent = AssistantAgent(
    name="basic_agent", model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"), 
    tools=[get_weather])

team = RoundRobinGroupChat(
    participants=[agent], termination_condition=TextMentionTermination("TERMINATE"))

team_result = await team.run(task="What is the weather in San Francisco?")

In [35]:
for message in team_result.messages:
    print(message.content)

What is the weather in San Francisco?
[FunctionCall(id='call_LCS4QBl0eQpVTGlnSFz5buby', arguments='{"city":"San Francisco"}', name='get_weather')]
[FunctionExecutionResult(content='The weather in San Francisco is 73 degrees and Sunny.', call_id='call_LCS4QBl0eQpVTGlnSFz5buby')]
The weather in San Francisco is currently 73 degrees and sunny.
TERMINATE


# Book Generator Team 

In this section, we will create a team of agents that can generate a simple childrens book. 
We will create 3 agents, 
- Planner: Receives a request to generate a book and creates a plan for the book content and structure 
- Image Generator: Has access to an image generation tool and can use it to generate images based on the text provided by the planner
- Book Generator: Wraps content created so far into a pdf book

In [ ]:
from fpdf import FPDF
import requests
import os
from tempfile import gettempdir
from PIL import Image
from io import BytesIO

def generate_and_save_pdf_report(sections: list, output_file: str = "book.pdf", report_title: str = "Book") -> str:
    """
    Generate a PDF report with text and images from provided sections.

    Args:
        sections (list): List of dictionaries containing section data. Each section should have:
            - title (str): Section title
            - content (str): Section content text
            - image (str): Path or URL to image file
            - level (str): Heading level (e.g., 'h1', 'h2')
        output_file (str, optional): Name of output PDF file. Defaults to "book.pdf".
        report_title (str, optional): Title of the report. Defaults to "Book".

    Returns:
        str: Path to the generated PDF file.

    Example:
        sections = [{
            'title': 'Chapter 1',
            'content': 'Chapter content...',
            'image': 'path/to/image.png',
            'level': 'h1'
        }]
        pdf_path = generate_and_save_pdf_report(sections, "output.pdf", "My Book")
    """
    
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    
    def sanitize_text(text):
        return text.encode('ascii', 'replace').decode()

    def add_image(img_path):
        try:
            # Remove sandbox prefix if present
            img_path = img_path.replace('sandbox:/', '')
            
            if img_path.startswith(('http://', 'https://')):
                response = requests.get(img_path)
                img = Image.open(BytesIO(response.content))
                temp_path = os.path.join(gettempdir(), "temp_image.png")
                img.save(temp_path, 'PNG')
                pdf.image(temp_path, x=10, w=190)
                os.remove(temp_path)
            else:
                pdf.image(img_path, x=10, w=190)
        except Exception as e:
            print(f"Image error: {e}")

    for section in sections:
        pdf.add_page()
        if section.get("title"):
            pdf.set_font('Arial', 'B', 16)
            pdf.cell(0, 10, txt=sanitize_text(section["title"]), ln=True)
        if section.get("content"):
            pdf.set_font('Arial', '', 12)
            pdf.multi_cell(0, 10, txt=sanitize_text(section["content"]))
        if section.get("image"):
            add_image(section["image"])
            pdf.ln(10)

    pdf.output(output_file)
    return output_file

from openai import  OpenAI 
# Define the tool functions
def generate_and_save_images(query: str, image_size: str = "1024x1024") -> List[str]:
    """   
    Function to generate images using OpenAI's DALL-E model based on a given prompt query.

    :param query: The prompt query to generate the image.
    :param image_size: The size of the image in the format "width x height". (default is "1024x1024")
    :return: A list of saved image file paths. 

    """
    client = OpenAI()
    response = client.images.generate(model="dall-e-3", prompt=query, n=1, size=image_size)
    saved_files = []
    
    if response.data:
        for image_data in response.data:
            file_name = str(uuid.uuid4()) + ".png"
            file_path = Path(file_name)
            img_url = image_data.url
            img_response = requests.get(img_url)
            if img_response.status_code == 200:
                with open(file_path, "wb") as img_file:
                    img_file.write(img_response.content)
                saved_files.append(str(file_path))
    return saved_files

sample_book = sections = [
    {
        "title": "Introduction - Early Life",
        "level": "h1",
        "image": "https://picsum.photos/536/354",
        "content": ("Marie Curie was born on 7 November 1867 in Warsaw, Poland. "
                    "She was the youngest of five children. Both of her parents were teachers. "
                    "Her father was a math and physics instructor, and her mother was the head of a private school. "
                    "Marie's curiosity and brilliance were evident from an early age."),
    }]
generate_and_save_pdf_report(sections=sample_book, output_file="sample_report.pdf", report_title="Marie Curie Report")

'sample_report.pdf'

In [37]:
# Story planner agent
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="A creative writer that can plan and write children's stories.",
    system_message="""You are a creative writer specialized in children's stories. 
    Your role is to create engaging, age-appropriate story outlines and content.
    For each story section, provide a title, content, and a detailed image description for illustration.
    Structure each section as:
    {
        'title': 'Section Title',
        'content': 'Story content for this section',
        'image_description': 'Detailed description for image generation'
    }"""
)

# Image generator agent
image_generator_agent = AssistantAgent(
    "image_generator_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that generates images for the story using DALL-E.",
    system_message="""You are an AI image generator specialist.
    Use the generate_and_save_images() function to create illustrations based on provided descriptions.""",
    tools=[generate_and_save_images]
)

# Book compiler agent
book_generator_agent = AssistantAgent(
    "book_generator_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4"),
    description="An agent that compiles the story and images into a PDF book.",
    system_message="""You are a book compilation specialist.
    Your role is to collect story sections and images, format them for PDF generation, and create the final book.
    IMPORTANT: Use the actual image file paths returned by the image generator, not placeholder names. For example if the image generator return '71e6aba5-1a7e-488c-9388-e3bc1eeb88c7.png', then use this exactly as the image path in the book generation without any prefix or suffix.
    Respond with 'TERMINATE' when the book is successfully generated.""",
    tools=[generate_and_save_pdf_report]
)


In [38]:

# Set up the group chat with termination condition
book_team = SelectorGroupChat(
    participants=[planner_agent, image_generator_agent, book_generator_agent],
    termination_condition=TextMentionTermination("TERMINATE"),
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    selector_prompt="You are  book generation cordinator. The following agent roles are helping you create a book: {roles}. Your goal is to read the progress so far and then select the next role from {participants} to take a turn. Only return the role. . {history}. You must call the agents in the right order  \nRead the above conversation.  Then select the next role from {participants} to play. Only return the role."
)
book_result = book_team.run_stream(task="Create a 1 page children's story with 2 images and text about the wonders of the amazon rainforest.") 

In [39]:

async for response in book_result:
    print(response)

source='user' models_usage=None content="Create a 1 page children's story with 2 images and text about the wonders of the amazon rainforest."
source='planner_agent' models_usage=RequestUsage(prompt_tokens=121, completion_tokens=611) content="{\n    'title': 'The Magical Amazon Adventure',\n    'content': 'Once upon a time, in a vibrant town nestled near the Amazon rainforest, lived a curious little girl named Mia. Mia loved nature and was fascinated by the stories her grandmother told her about the wonders of the Amazon. One sunny morning, Mia decided to venture into the jungle to see it for herself. Wearing her favourite green hat and a backpack filled with snacks, she set off on her adventure.\\n\\nAs Mia entered the rainforest, she was greeted by a symphony of sounds - chirping birds, buzzing insects, and rustling leaves. Brightly colored butterflies fluttered around her, dancing like little jewels in the air. “Wow!” Mia exclaimed. “This place is like a treasure chest!”\\n\\nMia mar

Selector selected the previous speaker: book_generator_agent


source='book_generator_agent' models_usage=RequestUsage(prompt_tokens=2188, completion_tokens=4) content='TERMINATE'
TaskResult(messages=[TextMessage(source='user', models_usage=None, content="Create a 1 page children's story with 2 images and text about the wonders of the amazon rainforest."), TextMessage(source='planner_agent', models_usage=RequestUsage(prompt_tokens=121, completion_tokens=611), content="{\n    'title': 'The Magical Amazon Adventure',\n    'content': 'Once upon a time, in a vibrant town nestled near the Amazon rainforest, lived a curious little girl named Mia. Mia loved nature and was fascinated by the stories her grandmother told her about the wonders of the Amazon. One sunny morning, Mia decided to venture into the jungle to see it for herself. Wearing her favourite green hat and a backpack filled with snacks, she set off on her adventure.\\n\\nAs Mia entered the rainforest, she was greeted by a symphony of sounds - chirping birds, buzzing insects, and rustling lea

In [40]:
sections = [
    {
        "title": "Introduction - Early Life",
        "level": "h1",
        "image": "https://picsum.photos/536/354",
        "content": ("Marie Curie was born on 7 November 1867 in Warsaw, Poland. "
                    "She was the youngest of five children. Both of her parents were teachers. "
                    "Her father was a math and physics instructor, and her mother was the head of a private school. "
                    "Marie's curiosity and brilliance were evident from an early age."),
    }]